# Welcome to bank_geocode demo notebook


##### The purpose of this notebook is to map every branch of every public sector bank except SBI in India and to analyse the sort of coverage each bank has.

-------------------------------------------------------------------------------------------------------------------

## Section 1: Getting data on bank branches

### Import pre-requisites

In [1]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import re
import time
from matplotlib import pyplot as plt
import folium
import warnings
from branca.element import Template, MacroElement
from utils import find_coord, trim
from map_legend import orig_template
%matplotlib inline
warnings.filterwarnings("ignore")


All data sources have been clearly mentioned and given due credit in the [Data](https://github.com/nrarunkaushik/bank_geocode/tree/main/Data) folder. <br>
Import the bank data and see it structure

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
data = pd.read_excel('./Data/MOF_Data.xlsx')
data.head(10)

### Bank data preprocessing

The data clearly has some preprocessing to be done:
- A few columns which are of no importance to this analysis are present and hence will be removed
- "Center" has additional information about type of administrative domain which will make it difficult to search in open street maps
- Searching for every branch individually and doing a URL request call for every address is both time consuming and memory&network intensive
- Hence pincodes will be extracted from addresses as much as possible and its geo location will be used here

In [ ]:
data.drop(['S.No.','Part1 Code', 'Part2 Code','Date of Open', 'AD Category', 'License No.', 'License Date'],axis=1,inplace=True)

center = Parallel(n_jobs=-1)(delayed(
    lambda x: re.split(r' \(', x)[0]
)(data.loc[row,'Center']) for row in data.index)
data['Center'] = center

pins = Parallel(n_jobs=-1)(delayed(
    lambda x: int(re.findall(r'([1-9]\d{2}\s{0,1}\d{3})',x)[-1].replace(" ", "")) if len(re.findall(r'([1-9]\d{2}\s{0,1}\d{3})',x)) > 0 else 0
)(data.loc[row,'Address']) for row in data.index)
data['pincode'] = pins

data['pincode'].apply(lambda x: len(str(x))).value_counts()

This shows that there are 135 observations for which pincode is missing. So geolocation for those observations has to be found ot using url requests to openstreetmap.

------------------------------------------------------------------------------------------------------------------
Cleaned data is saved for future use

In [ ]:
data.to_csv('./Outputs/docs/Bankdata.csv')

## Section 2: Getting geo locations for Indian pincodes

Data for pincodes with latitude and longitude are sourced from multiple locations and pooled to make a comprehensive database of indian pincodes with their geocode/geolocation (i.e, latitude and longitude).
<br>
These data were of course in need of preprocessing, data cleaning and data mining for those pincodes and their geolocation to be extracted.

In [ ]:
pindata = pd.read_csv('pincodes.csv')
pindata2 = pd.read_csv('pincodes2.csv')
pindata3 = pd.read_csv('pincodes3.csv')

pindata = pindata.loc[pindata['latitude'].dropna().index,]
pindata['officename'] = [trim(x) for x in list(pindata.officename.unique())]
pindata2['pincode'] = pindata2['key'].apply(lambda x: int(x[3:]))
pindata2 = pindata2[['pincode','place_name', 'admin_name1', 'latitude', 'longitude']]
pindata = pindata[['pincode','officename', 'circlename', 'latitude', 'longitude']]
pindata.columns = pindata2.columns
pindata = pindata.loc[~pindata['pincode'].isin(pindata2['pincode'])]
pindata = pd.concat([pindata2, pindata], ignore_index=True)

pindata3.columns = ['V'+str(i) for i in pindata3.columns]
pindata3_new = pindata3.loc[~pindata3['V1'].isin(pindata['pincode'])]
pindata3_new = pd.DataFrame([g[1].sort_values(by='V11',ascending=False).values[0] for g in pindata3_new.groupby('V1')],columns=pindata3_new.columns)
pindata3_new = pindata3_new[['V1','V2','V3','V9','V10']]
pindata3_new.columns = pindata.columns
pindata = pd.concat([pindata, pindata3_new], ignore_index=True)


 Once the data is cleaned, extracted and pooled, it is saved for future use. <br>
 ##### Disclaimer:
 Even with multiple data sources, there were some pincodes for which their geolocation was still missing.

In [ ]:
pindata.columns = ['pincode', 'place_name', 'state_name', 'latitude', 'longitude']
pindata.to_csv('./Outputs/docs/indian_pincode_data.csv',index=False)

## Section 3: Merging bank branch data with pincode data

- Bank data has pincode column with all other columns containing info on each branch. Pincode data also has pincode with columns for latitude and longitude.
- To get latitude and longitude for all branches in bank data for whose pincode had their geolocation in pincode data, bank data & pin data has to be left merged.

In [ ]:
joindata = pd.merge(data, pindata[['latitude', 'longitude', 'pincode']], on='pincode', how='left')
joindata.head(10)

In order to further analyse the data for which either:
- pincode is missing or
- pincode's geolocation is not found <br>

the merged data is split based on latitude missingness and the data subset for which it is missing has its lat/long removed

In [ ]:
latlong_old = joindata.loc[~np.isnan(joindata['latitude']),:]
latlong_miss = joindata.loc[np.isnan(joindata['latitude']),:]
latlong_miss['ix'] = latlong_miss.index
latlong_miss.drop(['latitude','longitude'],axis=1,inplace=True)


## Section 4: Finding lat/long for those it is mssing

### Using OSM
"find_coord" function from utils is used to do the URL request to find geocode for the lat/long-missing observations. The process is parallelized for efficiency gains in runtime. The function progressively checks for each of below in case earlier did not return anything:
1. Address
2. Bank name with branch location and center(city) with search filter for banks
3. Bank name with branch location and center(city)
4. Bank name with branch location and district with search filter for banks
5. Bank name with branch location and district
6. Branch, center, district, state
7. Center, district, state
8. District, state<br>

If no geocode found for any of the above searches, then NaNs are returned. Also after each search, it is checked whether the result gotten is any is within India's geographic bounding box.<br>
This search result is merged with data of lat/long missing.

In [ ]:
x1 = time.time()
coord = Parallel(n_jobs=-1,verbose=10)(delayed(find_coord)(joindata.loc[ix,:]) for ix in latlong_miss.index)
x2 = time.time()
print(x2-x1)

df = pd.DataFrame(coord, columns=['ix','lat','long'],dtype='float64')
df = df.astype({'ix':int})
df.columns = ['ix', 'latitude','longitude']

latlong_new = pd.merge(latlong_miss, df, on='ix', how='left')
latlong_new.drop(['ix'],axis=1,inplace=True)

latlong = pd.concat([latlong_old, latlong_new], ignore_index=True)

### Using State

However, there were still some observations for which lat/long were NaNs. As a final desparate attempt, those observations are coded with their state's geolocation. <br>
In order to do that, the data containing geolocations of state and union territories is cleaned and processed into required format.

In [ ]:
statepin = pd.read_csv('states.csv')
statepin['States'] = statepin.States.apply(lambda x: re.split(r' \(', x)[0].upper())

statepin.loc[statepin['States']=='ANDAMAN AND NICOBAR','States'] = 'ANDAMAN & NICOBAR ISLANDS'
statepin.loc[statepin['States']=='DELHI','States'] = 'NCT OF DELHI'

statepin = statepin.astype({'Latitude':float,'Longitude':float})

For each state-wise group of observations for which lat/long were missing, their state's geolocation is imputed.

In [ ]:
for state,group in latlong.loc[np.isnan(latlong['latitude']),:].groupby('State'):
    for ind in group.index:
        latlong.loc[ind,'latitude'] = statepin.loc[statepin['States']==state,'Latitude'].values[0]
        latlong.loc[ind,'longitude'] = statepin.loc[statepin['States']==state,'Longitude'].values[0]


The final data of all bank branches with their pincodes, laititude and longitude is saved for future use.

In [ ]:
latlong.to_csv('./Outputs/docs/latlong.csv',index=False)

## Section 5: Mapping (finally!)

Each branch is mapped using a different color on an interactive map created using Folium. In addition to that, an additional feature included here is the legend which shows branch numbers by bank and by state inside the map itself.

In [ ]:
colordict = {'UCO BANK':'black',
             'CENTRAL BANK OF INDIA':'green',
             'BANK OF INDIA':'lime',
             'CANARA BANK':'blue',
             'UNION BANK OF INDIA':'yellow', 
             'PUNJAB NATIONAL BANK':'gold',
             'BANK OF BARODA':'grey',
             'INDIAN BANK':'purple',
             'INDIAN OVERSEAS BANK':'orange',
             'BANK OF MAHARASHTRA':'maroon',
             'PUNJAB AND SIND BANK':'turquoise',
             'JAMMU & KASHMIR BANK LTD':'red'
            }

Map is created bank wise and in the legend, number of branches in each state is also shown for top 15 states and saved in Outputs folder

In [ ]:
for bank in colordict.keys():
    branches = folium.Map()
    datasub = latlong.loc[latlong['Bank']==bank,['State', 'District','Bank', 'Branch','latitude', 'longitude']]
    branches.fit_bounds([[6.46,68.11], [35.52, 97.4]])

    bank_div = datasub.groupby('Bank').count()['Branch'].sort_values(ascending=False)
    bank_str = ''
    for b in bank_div.index:
        bank_str += """<li><span style='background:%s;opacity:0.7;'></span>%s</li>"""%(colordict[b],b+'(%d)'%(bank_div[b]))

    state_div = datasub.groupby('State').count()['Branch']
    others = state_div.sum() - state_div.sort_values(ascending=False)[:14].sum()
    state_div = state_div.sort_values(ascending=False)[:14]
    if others != 0:
        state_div['Others'] = others
    state_str = ''
    for s in state_div.index:
        state_str += """<li><span style='background:linen;opacity:0.7;'></span>%s</li>"""%(s+'(%d)'%(state_div[s]))

    macro = MacroElement()
    template = orig_template
    template = template.replace('toreplacetitle',bank+" Branches")
    template = template.replace('toreplace1',bank_str)
    template = template.replace('toreplace2',state_str)
    macro._template = Template(template)
    
    if len(datasub) > 5000:
        datasub = datasub.drop_duplicates(subset=['latitude','longitude'])
    for i in datasub.values:
        folium.CircleMarker(location=[i[4],i[5]],
                            radius=2,
                            popup=i[3] + ', ' + i[1] + ', ' + i[0],
                            color='b',
                            key_on = i[2],
                            fill_color=colordict[i[2]],
                            fill=True,
                            fill_opacity=0.7
                           ).add_to(branches)
   
    
    branches.get_root().add_child(macro)
    branches.save('./Outputs/maps/Bankwise/'+bank+'.html')


Map is created state wise and in the legend, number of branches across each is also shown. All maps are saved in Outputs folder for easy access later.

In [ ]:
for state in latlong.State.unique():
    branches = folium.Map()
    datasub = latlong.loc[latlong['State']==state,['State', 'District','Bank', 'Branch','latitude', 'longitude']]
    branches.fit_bounds([[datasub.latitude.min(),datasub.longitude.min()], [datasub.latitude.max(), datasub.longitude.max()]])

    bank_div = datasub.groupby('Bank').count()['Branch'].sort_values(ascending=False)
    bank_str = ''
    for b in bank_div.index:
        bank_str += """<li><span style='background:%s;opacity:0.7;'></span>%s</li>"""%(colordict[b],b+'(%d)'%(bank_div[b]))

    state_div = datasub.groupby('State').count()['Branch']
    others = state_div.sum() - state_div.sort_values(ascending=False)[:14].sum()
    state_div = state_div.sort_values(ascending=False)[:14]
    if others != 0:
        state_div['Others'] = others
    state_str = ''
    for s in state_div.index:
        state_str += """<li><span style='background:linen;opacity:0.7;'></span>%s</li>"""%(s+'(%d)'%(state_div[s]))

    macro = MacroElement()
    template = orig_template
    template = template.replace('toreplacetitle',state+" Bank Branches")
    template = template.replace('toreplace1',bank_str)
    template = template.replace('toreplace2',state_str)
    macro._template = Template(template)
    if len(datasub) > 5000:
        datasub = datasub.drop_duplicates(subset=['latitude','longitude'])
    for i in datasub.values:
        folium.CircleMarker(location=[i[4],i[5]],
                            radius=2,
                            popup=i[3] + ', ' + i[1] + ', ' + i[0],
                            color='b',
                            key_on = i[2],
                            fill_color=colordict[i[2]],
                            fill=True,
                            fill_opacity=0.7
                           ).add_to(branches)
    
    branches.get_root().add_child(macro)
    branches.save('./Outputs/maps/Statewise/'+state+'.html')


Finally, a combined map is created for whole of India across all of the banks. 
##### Disclaimer:
Since this data is huge for an interactive map to handle, observations with lat/long duplicates are dropped

In [ ]:
branches = folium.Map()

branches.fit_bounds([[6.46,68.11], [35.52, 97.4]])

for i in latlong[['State', 'District','Bank', 'Branch','latitude', 'longitude']].drop_duplicates(subset=['latitude','longitude']).values:
    folium.CircleMarker(location=[i[4],i[5]],
                        radius=2,
                        popup=i[3] + ', ' + i[1] + ', ' + i[0],
                        color='b',
                        key_on = i[2],
                        fill_color=colordict[i[2]],
                        fill=True,
                        fill_opacity=0.7
                       ).add_to(branches)
bank_div = latlong.groupby('Bank').count()['Branch'].sort_values(ascending=False)
bank_str = ''
for b in bank_div.index:
    bank_str += """<li><span style='background:%s;opacity:0.7;'></span>%s</li>"""%(colordict[b],b+'(%d)'%(bank_div[b]))

state_div = latlong.groupby('State').count()['Branch']
others = state_div.sum() - state_div.sort_values(ascending=False)[:14].sum()
state_div = state_div.sort_values(ascending=False)[:14]
if others != 0:
    state_div['Others'] = others
state_str = ''
for s in state_div.index:
    state_str += """<li><span style='background:linen;opacity:0.7;'></span>%s</li>"""%(s+'(%d)'%(state_div[s]))

macro = MacroElement()
template = orig_template
template = template.replace('toreplacetitle',"Bank Branches of India")
template = template.replace('toreplace1',bank_str)
template = template.replace('toreplace2',state_str)
macro._template = Template(template)
branches.get_root().add_child(macro)
branches.save('./Outputs/maps/OVERALL.html')

Run this cell below to see the interactive map of branches across all states and banks. It is not shown here due to its increase in file size of this Jupyter notebook so a screenshot of it is included here

In [ ]:
branches

In [ ]:
from IPython.display import display
from PIL import Image
img_PIL = Image.open(r'./Outputs/maps/overall.png')
display(img_PIL)

## Section 6: Saving results

Plotting in map is fun but getting to know the numbers across states and banks is also necessary. <br>
Hence, the following are created:
1. Count of branches grouped by 'State','Bank' and 'Population Group
2. No. of branches across banks
3. No. of branches across states
4. No. of branches across banks segregated by area type
5. No. of branches across states segregated by area type<br>

Area type/Population group divides the locality into Rural, semi urban, urban and metropolitan categories.


In [ ]:
branch_count = pd.DataFrame([(i[0],i[1],i[2], g.count()['Branch']) for i, g in latlong.groupby(['State','Bank','Population Group'])], columns=['State','Bank','Area_type','#Branches'])
branches_statewise = branch_count.groupby('State').sum().sort_values('#Branches',ascending=False)
branches_bankwise = branch_count.groupby('Bank').sum().sort_values('#Branches',ascending=False)

In [ ]:
bank_areawise = pd.crosstab(branch_count['Bank'],branch_count['Area_type'],branch_count['#Branches'],aggfunc=sum)
bank_areawise['Total'] = bank_areawise.sum(1)
bank_areawise.sort_values('Total',ascending=False,inplace=True)
bank_areawise = bank_areawise[['Rural', 'Semi-urban', 'Urban','Metropolitan']]
bank_areawise.plot(kind='bar',stacked=True, figsize=(15,10), title="Bank branches across banks by Area type", fontsize=10)
plt.savefig('./Outputs/plots/Branches_across_banks.png',orientation='landscape',transparent=True,bbox_inches='tight')

In [ ]:
state_areawise = pd.crosstab(branch_count['State'],branch_count['Area_type'],branch_count['#Branches'],aggfunc=sum)
state_areawise['Total'] = state_areawise.sum(1)
state_areawise.sort_values('Total',ascending=False,inplace=True)
state_areawise = state_areawise[['Rural', 'Semi-urban', 'Urban','Metropolitan']]
state_areawise.plot(kind='bar',stacked=True, figsize=(15,10), title="Bank branches over all states by Area type", fontsize=10)
plt.savefig('./Outputs/plots/Branches_across_states.png',orientation='landscape',transparent=True,bbox_inches='tight')

In [ ]:
branch_count.to_csv('./Outputs/docs/Branches_count.csv',index=False)
branches_statewise.to_csv('./Outputs/docs/Branches_statewise.csv')
branches_bankwise.to_csv('./Outputs/docs/Branches_bankwise.csv')
bank_areawise.to_csv('./Outputs/docs/Branches_acrossAreatypes_forallBanks.csv')
state_areawise.to_csv('./Outputs/docs/Branches_acrossAreatypes_forallStates.csv')
